# **<mark>Dynamic SQL</mark>**

**Dynamic SQL** là một kỹ thuật lập trình cho phép bạn tạo các câu lệnh SQL động trong thời gian chạy. Nó cho phép bạn tạo câu lệnh SQL có mục đích chung và linh hoạt hơn vì toàn bộ văn bản của câu lệnh SQL có thể không được biết khi biên dịch. Ví dụ: bạn có thể sử dụng SQL động để tạo một thủ tục được lưu trữ truy vấn dữ liệu đối với một bảng có tên không được biết cho đến thời gian chạy.

Tạo một SQL động rất đơn giản, bạn chỉ cần đặt nó thành một chuỗi như sau:

```
'SELECT * FROM production.products';
```
Để thực thi một câu lệnh SQL động, bạn gọi thủ tục được lưu trữ sp_executesql như được hiển thị trong câu lệnh sau:

In [ ]:
EXEC sp_executesql N'SELECT * FROM production.products';

<mark>Bởi vì sp\_executesql chấp nhận SQL động dưới dạng một chuỗi Unicode, bạn cần đặt tiền tố nó bằng N.</mark>

### **<mark>Sử dụng SQL động để truy vấn từ bất kỳ ví dụ bảng nào</mark>**

Đầu tiên, khai báo hai biến, @table để giữ tên của bảng mà bạn muốn truy vấn và @sql để giữ SQL động.

```
DECLARE 
    @table NVARCHAR(128),
    @sql NVARCHAR(MAX);

```

Thứ hai, đặt giá trị của biến @table thành production.products.

```
SET @table = N'production.products';

```

Thứ ba, xây dựng SQL động bằng cách nối câu lệnh SELECT với tham số tên bảng:

```
SET @sql = N'SELECT * FROM ' + @table;

```

Thứ tư, gọi thủ tục được lưu trữ sp\_executesql bằng cách truyền tham số @sql.  

```
EXEC sp_executesql @sql;

```

In [ ]:
DECLARE 
    @table NVARCHAR(128),
    @sql NVARCHAR(MAX);

SET @table = N'production.products';

SET @sql = N'SELECT * FROM ' + @table;

EXEC sp_executesql @sql;

Để truy vấn dữ liệu từ một bảng khác, bạn thay đổi giá trị của biến @table. Tuy nhiên, sẽ thực tế hơn nếu chúng ta bọc khối T-SQL ở trên trong một quy trình được lưu trữ.

### **<mark>SQL động và stored procedures</mark>**

In [1]:
CREATE PROC usp_query (
    @table NVARCHAR(128)
)
AS
BEGIN

    DECLARE @sql NVARCHAR(MAX);
    -- construct SQL
    SET @sql = N'SELECT * FROM ' + @table;
    -- execute the SQL
    EXEC sp_executesql @sql;
    
END;

Commands completed successfully.

Total execution time: 00:00:00

In [2]:
EXEC usp_query 'production.brands';

(9 rows affected)

Total execution time: 00:00:00.013

brand_id,brand_name
1,Electra
2,Haro
3,Heller
4,Pure Cycles
5,Ritchey
6,Strider
7,Sun Bicycles
8,Surly
9,Trek


In [3]:
CREATE OR ALTER PROC usp_query_topn(
    @table NVARCHAR(128),
    @topN INT,
    @byColumn NVARCHAR(128)
)
AS
BEGIN
    DECLARE 
        @sql NVARCHAR(MAX),
        @topNStr NVARCHAR(MAX);

    SET @topNStr  = CAST(@topN as nvarchar(max));

    -- construct SQL
    SET @sql = N'SELECT TOP ' +  @topNStr  + 
                ' * FROM ' + @table + 
                    ' ORDER BY ' + @byColumn + ' DESC';
    -- execute the SQL
    EXEC sp_executesql @sql;
    
END;

Commands completed successfully.

Total execution time: 00:00:00.009

In [7]:
EXEC usp_query_topn 
        'production.products',
        5, 
        'model_year';

(5 rows affected)

Total execution time: 00:00:00.007

product_id,product_name,brand_id,category_id,model_year,list_price
316,Trek Checkpoint ALR 4 Women's - 2019,9,7,2019,1699.99
317,Trek Checkpoint ALR 5 - 2019,9,7,2019,1999.99
318,Trek Checkpoint ALR 5 Women's - 2019,9,7,2019,1999.99
319,Trek Checkpoint SL 5 Women's - 2019,9,7,2019,2799.99
320,Trek Checkpoint SL 6 - 2019,9,7,2019,3799.99


In [5]:
EXEC usp_query_topn 
        'production.stocks',
        10, 
        'quantity';

(10 rows affected)

Total execution time: 00:00:00.004

store_id,product_id,quantity
1,30,30
1,61,30
1,64,30
1,68,30
1,106,30
1,109,30
1,188,30
1,193,30
1,219,30
1,292,30


### **<mark>SQL động và SQL Injection</mark>**

In [8]:
CREATE TABLE sales.tests(id INT);

Commands completed successfully.

Total execution time: 00:00:00.010

In [9]:
EXEC usp_query 'production.brands';

(9 rows affected)

Total execution time: 00:00:00.003

brand_id,brand_name
1,Electra
2,Haro
3,Heller
4,Pure Cycles
5,Ritchey
6,Strider
7,Sun Bicycles
8,Surly
9,Trek


In [ ]:
CREATE PROC usp_query (
    @table NVARCHAR(128)
)
AS
BEGIN

    DECLARE @sql NVARCHAR(MAX);
    -- construct SQL
    SET @sql = N'SELECT * FROM ' + @table;
    -- execute the SQL
    EXEC sp_executesql @sql;
    
END;

In [ ]:
EXEC usp_query 'production.brands;DROP TABLE sales.tests';

Kỹ thuật này được gọi là SQL injection. Khi câu lệnh được thực thi, bảng sales.tests sẽ bị loại bỏ, vì thủ tục được lưu trữ usp\_query thực thi cả hai câu lệnh:

In [ ]:
SELECT * FROM production.brands;DROP TABLE sales.tests

SQL Injection: [SQL Injection - Start (hacksplaining.com)](https://www.hacksplaining.com/exercises/sql-injection#/start)

Để ngăn chặn việc đưa vào SQL này, bạn có thể sử dụng hàm QUOTENAME () như được hiển thị trong truy vấn sau:

QUOTENAME: [QUOTENAME (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/t-sql/functions/quotename-transact-sql?view=sql-server-ver16)

In [ ]:
CREATE OR ALTER PROC usp_query
(
    @schema NVARCHAR(128), 
    @table  NVARCHAR(128)
)
AS
    BEGIN
        DECLARE 
            @sql NVARCHAR(MAX);
        -- construct SQL
        SET @sql = N'SELECT * FROM ' 
            + QUOTENAME(@schema) 
            + '.' 
            + QUOTENAME(@table);
        -- execute the SQL
        EXEC sp_executesql @sql;
    END;

In [ ]:
EXEC usp_query 'production','brands';

In [ ]:
'production'.'brands'

In [ ]:
EXEC usp_query 
        'production',
        'brands;DROP TABLE sales.tests';

### **<mark>Thông tin thêm về thủ tục lưu trữ sp\_executesql</mark>**

```
EXEC sp_executesql 
    sql_statement  
    parameter_definition
    @param1 = value1,
    @param2 = value2,
    ...

```

Trong cú pháp này:

- sql\_statement là một chuỗi Unicode chứa câu lệnh T-SQL. Câu lệnh sql\_statement có thể chứa các tham số như SELECT \* FROM table\_name WHERE id = @ id 
- parameter\_definition là một chuỗi chứa định nghĩa của tất cả các tham số được nhúng trong câu lệnh sql\_statement. Mỗi định nghĩa tham số bao gồm tên tham số và kiểu dữ liệu của nó, ví dụ: @id INT. Các định nghĩa tham số được phân tách bằng dấu phẩy (,). 
- @ param1 = value1, @ param2 = value2,… chỉ định giá trị cho mọi tham số được xác định trong chuỗi parameter\_definition.

In [ ]:
EXEC sp_executesql
    N'SELECT *
        FROM 
            production.products 
        WHERE 
            list_price> @listPrice AND
            category_id = @categoryId
        ORDER BY
            list_price DESC' 
    ,N'@listPrice DECIMAL(10,2),
    @categoryId INT'
    ,@listPrice = 100
    ,@categoryId = 1;